# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [3]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df['col2'] = pd.Series([2,3,4,5])
df

,col1,col2
0,1,2
1,2,3
2,3,4
3,4,5


## 2. Deleting a row in a DataFrame

In [11]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df.drop('d', inplace = True) # return new object
df

,col1
a,1
b,2
c,3


## 3. Creating a DataFrame from a few Series

In [14]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))
df = pd.DataFrame(data = {'ser_1':ser_1,'ser_2':ser_2,'ser_3':ser_3})
df

,ser_1,ser_2,ser_3
0,1.584393,1.671409,0.792153
1,1.532483,-0.724600,-1.761747
2,-0.843201,-0.157244,1.712914
3,0.857407,-0.237762,0.809166
4,-1.291569,-2.093547,-1.173451
5,0.641332,-1.422062,0.667052


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [22]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df.loc[:,'col_2']
df['col_2']

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [32]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.loc['obs3']
df.loc['obs3',:]

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [33]:
# using the same DataFrame, index into into its first row
df.iloc[0]
df.iloc[0,:]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [28]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'])

/Applications/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py:755: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())
/Applications/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py:755: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


## 2. How to load the training and testing subsets

In [35]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0, encoding='latin-1')
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0, encoding='latin-1')

In [36]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [37]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [38]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [39]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [41]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [43]:
print('RMSE for my estimate function: %s' % evaluate(my_estimate_func))

RMSE for my estimate function: 1.2323719526527521


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [48]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    # first, index into all ratings of this movie
    # second, compute the mean of those ratings
    # 
    movie_condition = movielens_train.movie_id == movie_id
    ratings = movielens_train.loc[movie_condition, 'rating']
    if ratings.empty:
        return 3.0
    else:
        return ratings.mean() 
    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)
print('RMSE for estimate1: %s' % evaluate(collab_mean))

RMSE for estimate1: 1.1234279896011794


# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [51]:
user_info = users.set_index('user_id')

def collab_zip(user_id, movie_id):
    # first, index into all ratings of this movie
    # second, compute the mean of those ratings
    # 
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means_by_zip = ratings_by_others.pivot_table('rating', index='movie_id', columns='zip')
    user_zip = user_info.loc[user_id, 'zip']
    if user_zip in means_by_zip.columns: 
        return means_by_zip.loc[movie_id, user_zip]
    else:
        return means_by_zip.loc[movie_id].mean()

print('RMSE for collab_gender: %s' % evaluate(collab_zip))

RMSE for collab_gender: 1.1256640319176328


# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine